In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import traceback

In [2]:
df = pd.read_csv(r'C:\Users\user\boyu571_github\01_Kakaobank_SKKU_Research_23\eng_news_nocontent.csv')
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
0,"{'id': None, 'name': 'Trendmicro.com'}",Trend Micro,REDBALDKNIGHT/BRONZE BUTLER’s Daserf Backdoor ...,"REDBALDKNIGHT, also known as BRONZE BUTLER and...",https://blog.trendmicro.com/trendlabs-security...,https://blog.trendmicro.com/trendlabs-security...,2017-11-07T12:34:44Z,by Joey Chen and MingYen Hsieh (Threat Analyst...,Trendmicro.com
1,"{'id': None, 'name': 'Computerworld'}","Lucas Mearian, Lucas Mearian",What is blockchain? The most disruptive tech i...,Blockchain is poised to change IT in much the ...,https://www.computerworld.com/article/3191077/...,https://images.techhive.com/images/article/201...,2017-11-08T01:06:00Z,Blockchain is poised to change IT in much the ...,Computerworld
2,"{'id': None, 'name': 'hackernoon.com'}",Yaoqi Jia,Demystifying Hashgraph: Benefits and Challenges,"Recently, on our Zilliqa social channels a num...",https://hackernoon.com/demystifying-hashgraph-...,NaN,2017-11-08T17:21:35Z,"Recently, on our Zilliqa social channels a num...",hackernoon.com
3,"{'id': None, 'name': 'Topix.com'}",NaN,BitTorrent inventor announces eco-friendly Bit...,A Bitcoin transaction wastes as much electrici...,http://www.topix.com/tech/p2p/2017/11/bittorre...,http://picproxy.topixcdn.com/pics/up-,2017-11-08T23:21:17Z,A Bitcoin transaction wastes as much electrici...,Topix.com
4,"{'id': None, 'name': 'Ml-implode.com'}",NaN,Bitcoin Fork Called Off: Prices Soar After Seg...,The price of a bitcoin went on a tear on Wedne...,http://ml-implode.com/staticnews/2017-11-09_Bi...,NaN,2017-11-09T00:00:00Z,The price of a bitcoin went on a tear on Wedne...,Ml-implode.com
...,...,...,...,...,...,...,...,...,...
744427,"{'id': None, 'name': 'Biztoc.com'}",u.today,"Negatively Correlates With Bitcoin (BTC), What...",Shiba Inu continues to focus on utility as its...,https://biztoc.com/x/8248d6342c37d743,https://c.biztoc.com/p/8248d6342c37d743/s.webp,2023-09-30T21:50:10Z,Shiba Inu continues to focus on utility as its...,Biztoc.com
744428,"{'id': None, 'name': 'Slashdot.org'}",feedfeeder,Industry leaders and policymakers weigh in on ...,Industry leaders and policymakers weigh in on ...,https://slashdot.org/firehose.pl?op=view&amp;i...,NaN,2023-09-30T21:53:20Z,Sign up for the Slashdot newsletter! OR check ...,Slashdot.org
744429,"{'id': None, 'name': 'Investing.com'}",U.Today,Shiba Inu (SHIB) Negatively Correlates With Bi...,Shiba Inu (SHIB) Negatively Correlates With Bi...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/LYNXNPE...,2023-09-30T22:00:22Z,U.Today - Dog-themed cryptocurrency is seeing ...,Investing.com
744430,"{'id': None, 'name': 'Ambcrypto.com'}",Benjamin Njiri,Can NEAR stay above $1?,"NEAR could be ripe for a price reversal, but t...",https://ambcrypto.com/?p=347645,https://statics.ambcrypto.com/wp-content/uploa...,2023-09-30T22:30:04Z,Disclaimer: The information presented does not...,Ambcrypto.com


In [123]:
df[df["press"] == "Toronto Star"]["url"].iloc[10]

'https://www.thestar.com/business/tech_news/2017/12/07/bitcoin-surges-past-16000-for-first-time-as-hack-raises-concerns-ahead-of-us-trade.html'

In [125]:
TorontoStar = df[df['press'] == "Toronto Star"]
TorontoStar

,source,author,title,description,url,urlToImage,publishedAt,content,press
241,"{'id': None, 'name': 'Toronto Star'}",Annie Massa - Bloomberg,Startup figures out how to put tomatoes on blo...,"System can track tomato ripeness, colour, flav...",https://www.thestar.com/business/2017/11/10/st...,https://www.thestar.com/content/dam/thestar/bu...,2017-11-10T15:00:00Z,Intrigued by how blockchain was changing finan...,Toronto Star
2730,"{'id': None, 'name': 'Toronto Star'}",Aleksandra Sagan - The Canadian Press,How hackers use cryptojacking to help ‘mine’ f...,Internet sleuths have discovered malicious cod...,https://www.thestar.com/business/2017/11/19/ho...,https://www.thestar.com/content/dam/thestar/bu...,2017-11-19T17:05:45Z,Web surfers have spotted such code on major we...,Toronto Star
4620,"{'id': None, 'name': 'Toronto Star'}",NATHANIEL POPPER - The New York Times,Another large bitcoin exchange draws warnings,Concerns over yet another exchange for the pop...,https://www.thestar.com/business/wealth/2017/1...,https://www.thestar.com/content/dam/thestar/bu...,2017-11-25T16:00:00Z,“I’m worried about the systemic risk that this...,Toronto Star
5242,"{'id': None, 'name': 'Toronto Star'}","Suzy Waite - Bloomberg,Nishant Kumar","Bitcoin soars toward $10,000 U.S. mark despite...","Bitcoin blew past $9,700 just a week after top...",https://www.thestar.com/business/tech_news/201...,https://www.thestar.com/content/dam/thestar/bu...,2017-11-27T16:33:12Z,Bitcoin’s march toward respectability faces an...,Toronto Star
5857,"{'id': None, 'name': 'Toronto Star'}","Nour Al Ali - Bloomberg,Christopher Kingdon",Elon Musk shoots down theory he’s Bitcoin’s my...,Tesla CEO Elon Musk tweeted a response to a bl...,https://www.thestar.com/business/2017/11/28/el...,https://www.thestar.com/content/dam/thestar/bu...,2017-11-28T17:17:55Z,DUBAI—Not only does Elon Musk deny being the m...,Toronto Star
...,...,...,...,...,...,...,...,...,...
705144,"{'id': None, 'name': 'Toronto Star'}",Thea Gribilas - Staff Reporter,Toronto Hydro warning customers of scammers th...,The fraudsters are contacting customers by pho...,https://www.thestar.com/news/gta/2023/07/07/to...,https://images.thestarimages.com/cuyJ5SrSL0UL2...,2023-07-07T16:39:09Z,Toronto Hydro is warning customers about ongoi...,Toronto Star
706003,"{'id': None, 'name': 'Toronto Star'}",Meg Kinnard - The Associated Press,"John Warren, ex-South Carolina GOP gubernatori...","COLUMBIA, S.C. (AP) — John Warren — the multim...",https://www.thestar.com/news/world/us/2023/07/...,https://images.thestarimages.com/A89OF5SqjvvR0...,2023-07-10T15:05:51Z,"COLUMBIA, S.C. (AP) John Warren the multimilli...",Toronto Star
707941,"{'id': None, 'name': 'Toronto Star'}",Terry Spencer - The Associated Press,"Andrew Tate sues Florida woman, saying her lie...","FORT LAUDERDALE, Fla. (AP) — Controversial soc...",https://www.thestar.com/news/world/us/2023/07/...,https://images.thestarimages.com/ZpU77O425r3Zo...,2023-07-13T18:13:37Z,"FORT LAUDERDALE, Fla. (AP) Controversial socia...",Toronto Star
708675,"{'id': None, 'name': 'Toronto Star'}",Megan Janetsky - The Associated Press,A wave of political turbulence is rolling thro...,MEXICO CITY (AP) — Central America is experien...,https://www.thestar.com/news/world/americas/20...,https://images.thestarimages.com/QzcF1eg8BI5Ac...,2023-07-14T21:20:28Z,MEXICO CITY (AP) Central America is experienci...,Toronto Star


In [126]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import traceback

def get_contents(yahoo):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for url in tqdm(yahoo['url']):
        try:
            response = requests.get(url, headers=headers, timeout=60)  # 1분 timeout 설정
            soup = BeautifulSoup(response.text, "html.parser")

            # Modified this part to only get the first 'div+ article' element
            article_elements = soup.select('#article-body')
            
            
            # if not article_elements:
            #     article_elements = soup.select('#contentdata')
                       
            if article_elements:
                content = article_elements[0].get_text()
            else:
                content = ''

            contents.append(content)
        except (requests.Timeout, Exception) as e:
            if isinstance(e, requests.Timeout):
                print(f"Request timed out for URL: {url}")
            else:
                print(f"Error occurred: {e}")
                traceback.print_exc()
            contents.append('')

    return contents


In [73]:
# from tqdm import tqdm
# import requests
# from bs4 import BeautifulSoup
# import traceback

# def get_contents(yahoo):
#     contents = []

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

#     for url in tqdm(yahoo['url']):
#         try:
#             response = requests.get(url, headers=headers, timeout=60)  # 1분 timeout 설정
#             soup = BeautifulSoup(response.text, "html.parser")

#             # 첫 번째 선택자로부터 내용 추출
#             start_content_elements = soup.select('.mainArea .FirstEle p p')
#             start_content = start_content_elements[0].get_text() if start_content_elements else ''

#             # 두 번째 선택자로부터 내용 추출
#             article_elements = soup.select('.paywall > p')
#             article_content = ' '.join([ele.get_text() for ele in article_elements])

#             # 두 내용을 결합
#             combined_content = start_content + ' ' + article_content

#             contents.append(combined_content)
#         except (requests.Timeout, Exception) as e:
#             if isinstance(e, requests.Timeout):
#                 print(f"Request timed out for URL: {url}")
#             else:
#                 print(f"Error occurred: {e}")
#                 traceback.print_exc()
#             contents.append('')

#     return contents


In [121]:
url = 'https://www.cryptopolitan.com/uniswap-price-prediction/'
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
response = requests.get(url, headers=headers)
print(response.status_code)
# soup = BeautifulSoup(response.text, "html.parser")
# soup

403


In [25]:
# from tqdm import tqdm
# import requests
# from bs4 import BeautifulSoup

# def get_contents(yahoo):
#     contents = []

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

#     for url in tqdm(yahoo['url']):
#         try:
#             response = requests.get(url, headers=headers)
#             soup = BeautifulSoup(response.text, "html.parser")
            
#             selected_elements = soup.select('.ccn-content')
            
#             # soup.select('.ccn-content')가 빈 리스트를 반환할 경우 에러 메시지 출력
#             if not selected_elements:
#                 print(f"No elements found for URL {url}.")
#                 contents.append('')
#                 continue

#             content = []
#             for element in selected_elements:
#                 content.append(element.get_text())
#                 print(element)

#             cleaned_content = ' '.join(content)
#             contents.append(cleaned_content)
#         except Exception as e:
#             print(f"Error occurred while processing URL {url}: {e}")
#             contents.append('')

#     return contents



In [88]:
# import requests
# from bs4 import BeautifulSoup
# import re
# from tqdm import tqdm

# def get_contents(yahoo):
#     contents = []

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

#     for url in tqdm(yahoo['url']):
#         try:
#             response = requests.get(url, headers=headers)
#             soup = BeautifulSoup(response.text, "html.parser")

#             # 모든 클래스 이름 중에서 ".col-*-offset-1" 패턴을 가진 것들을 찾는다
#             classes = set()
#             for tag in soup.find_all(True, {'class': True}):
#                 for class_name in tag['class']:
#                     if re.match(r'col-\w+-offset-1', class_name):
#                         classes.add(class_name)

#             # 해당 패턴을 가진 클래스들로부터 내용을 추출한다
#             content = []
#             for class_name in classes:
#                 content.extend(soup.select(f".{class_name}"))
            
#             cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content))
#             contents.append(cleaned_content)
#         except:
#             contents.append('')

#     return contents

In [127]:
head = TorontoStar.head()
head['content_full'] = get_contents(head)
head

100%|██████████| 5/5 [00:12<00:00,  2.58s/it]
<ipython-input-127-64d2931fe329>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  head['content_full'] = get_contents(head)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
241,"{'id': None, 'name': 'Toronto Star'}",Annie Massa - Bloomberg,Startup figures out how to put tomatoes on blo...,"System can track tomato ripeness, colour, flav...",https://www.thestar.com/business/2017/11/10/st...,https://www.thestar.com/content/dam/thestar/bu...,2017-11-10T15:00:00Z,Intrigued by how blockchain was changing finan...,Toronto Star,\n\nIntrigued by how blockchain was changing f...
2730,"{'id': None, 'name': 'Toronto Star'}",Aleksandra Sagan - The Canadian Press,How hackers use cryptojacking to help ‘mine’ f...,Internet sleuths have discovered malicious cod...,https://www.thestar.com/business/2017/11/19/ho...,https://www.thestar.com/content/dam/thestar/bu...,2017-11-19T17:05:45Z,Web surfers have spotted such code on major we...,Toronto Star,\n\nVANCOUVER—Anyone casually surfing the inte...
4620,"{'id': None, 'name': 'Toronto Star'}",NATHANIEL POPPER - The New York Times,Another large bitcoin exchange draws warnings,Concerns over yet another exchange for the pop...,https://www.thestar.com/business/wealth/2017/1...,https://www.thestar.com/content/dam/thestar/bu...,2017-11-25T16:00:00Z,“I’m worried about the systemic risk that this...,Toronto Star,\n\nSAN FRANCISCO — As the price of Bitcoin ha...
5242,"{'id': None, 'name': 'Toronto Star'}","Suzy Waite - Bloomberg,Nishant Kumar","Bitcoin soars toward $10,000 U.S. mark despite...","Bitcoin blew past $9,700 just a week after top...",https://www.thestar.com/business/tech_news/201...,https://www.thestar.com/content/dam/thestar/bu...,2017-11-27T16:33:12Z,Bitcoin’s march toward respectability faces an...,Toronto Star,\n\nBitcoin’s march toward respectability face...
5857,"{'id': None, 'name': 'Toronto Star'}","Nour Al Ali - Bloomberg,Christopher Kingdon",Elon Musk shoots down theory he’s Bitcoin’s my...,Tesla CEO Elon Musk tweeted a response to a bl...,https://www.thestar.com/business/2017/11/28/el...,https://www.thestar.com/content/dam/thestar/bu...,2017-11-28T17:17:55Z,DUBAI—Not only does Elon Musk deny being the m...,Toronto Star,\n\nDUBAI—Not only does Elon Musk deny being t...


In [128]:
# Adding the content to the DataFrame
TorontoStar['content_full'] = get_contents(TorontoStar)
TorontoStar

100%|██████████| 1271/1271 [55:15<00:00,  2.61s/it]
<ipython-input-128-33eb88a5405d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TorontoStar['content_full'] = get_contents(TorontoStar)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
241,"{'id': None, 'name': 'Toronto Star'}",Annie Massa - Bloomberg,Startup figures out how to put tomatoes on blo...,"System can track tomato ripeness, colour, flav...",https://www.thestar.com/business/2017/11/10/st...,https://www.thestar.com/content/dam/thestar/bu...,2017-11-10T15:00:00Z,Intrigued by how blockchain was changing finan...,Toronto Star,\n\nIntrigued by how blockchain was changing f...
2730,"{'id': None, 'name': 'Toronto Star'}",Aleksandra Sagan - The Canadian Press,How hackers use cryptojacking to help ‘mine’ f...,Internet sleuths have discovered malicious cod...,https://www.thestar.com/business/2017/11/19/ho...,https://www.thestar.com/content/dam/thestar/bu...,2017-11-19T17:05:45Z,Web surfers have spotted such code on major we...,Toronto Star,\n\nVANCOUVER—Anyone casually surfing the inte...
4620,"{'id': None, 'name': 'Toronto Star'}",NATHANIEL POPPER - The New York Times,Another large bitcoin exchange draws warnings,Concerns over yet another exchange for the pop...,https://www.thestar.com/business/wealth/2017/1...,https://www.thestar.com/content/dam/thestar/bu...,2017-11-25T16:00:00Z,“I’m worried about the systemic risk that this...,Toronto Star,\n\nSAN FRANCISCO — As the price of Bitcoin ha...
5242,"{'id': None, 'name': 'Toronto Star'}","Suzy Waite - Bloomberg,Nishant Kumar","Bitcoin soars toward $10,000 U.S. mark despite...","Bitcoin blew past $9,700 just a week after top...",https://www.thestar.com/business/tech_news/201...,https://www.thestar.com/content/dam/thestar/bu...,2017-11-27T16:33:12Z,Bitcoin’s march toward respectability faces an...,Toronto Star,\n\nBitcoin’s march toward respectability face...
5857,"{'id': None, 'name': 'Toronto Star'}","Nour Al Ali - Bloomberg,Christopher Kingdon",Elon Musk shoots down theory he’s Bitcoin’s my...,Tesla CEO Elon Musk tweeted a response to a bl...,https://www.thestar.com/business/2017/11/28/el...,https://www.thestar.com/content/dam/thestar/bu...,2017-11-28T17:17:55Z,DUBAI—Not only does Elon Musk deny being the m...,Toronto Star,\n\nDUBAI—Not only does Elon Musk deny being t...
...,...,...,...,...,...,...,...,...,...,...
705144,"{'id': None, 'name': 'Toronto Star'}",Thea Gribilas - Staff Reporter,Toronto Hydro warning customers of scammers th...,The fraudsters are contacting customers by pho...,https://www.thestar.com/news/gta/2023/07/07/to...,https://images.thestarimages.com/cuyJ5SrSL0UL2...,2023-07-07T16:39:09Z,Toronto Hydro is warning customers about ongoi...,Toronto Star,\n\nToronto Hydro is warning customers about o...
706003,"{'id': None, 'name': 'Toronto Star'}",Meg Kinnard - The Associated Press,"John Warren, ex-South Carolina GOP gubernatori...","COLUMBIA, S.C. (AP) — John Warren — the multim...",https://www.thestar.com/news/world/us/2023/07/...,https://images.thestarimages.com/A89OF5SqjvvR0...,2023-07-10T15:05:51Z,"COLUMBIA, S.C. (AP) John Warren the multimilli...",Toronto Star,"\n\nCOLUMBIA, S.C. (AP) — John Warren — the mu..."
707941,"{'id': None, 'name': 'Toronto Star'}",Terry Spencer - The Associated Press,"Andrew Tate sues Florida woman, saying her lie...","FORT LAUDERDALE, Fla. (AP) — Controversial soc...",https://www.thestar.com/news/world/us/2023/07/...,https://images.thestarimages.com/ZpU77O425r3Zo...,2023-07-13T18:13:37Z,"FORT LAUDERDALE, Fla. (AP) Controversial socia...",Toronto Star,"\n\nFORT LAUDERDALE, Fla. (AP) — Controversial..."
708675,"{'id': None, 'name': 'Toronto Star'}",Megan Janetsky - The Associated Press,A wave of political turbulence is rolling thro...,MEXICO CITY (AP) — Central America is experien...,https://www.thestar.com/news/world/americas/20...,https://images.thestarimages.com/QzcF1eg8BI5Ac...,2023-07-14T21:20:28Z,MEXICO CITY (AP) Central America is experienci...,Toronto Star,\n\nMEXICO CITY (AP) — Central America is expe...


In [107]:
head['content_full'].iloc[1]

'Investment manias throughout the centuries have ranged from tulips to tech stocks to housing; is bitcoin different? Illustration/Video: Daniel EpsteinFirst it was tulips. More recently it was tech stocks and then the housing market. Lately, it’s all about bitcoin.All four share a common denominator: They suffered through financial bubbles, or bubble-like conditions. The first three ended badly. The jury is still out on what’s in store for bitcoin.'

In [108]:
head['url'].iloc[0]

'http://www.wsj.com/video/bitcoin-the-worlds-most-dramatic-bubble-ever/518F37A8-B7B4-43AC-869E-6F0B3568EAB6.html'

In [122]:
with2 = empty_rows[empty_rows['content_full'] != '']
with2

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
655,"{'id': None, 'name': 'Moneycontrol'}",NaN,News Live: PSBs want large banks to lend to co...,This blog will keep track of key global and lo...,http://www.moneycontrol.com/news/business/news...,http://static-news.moneycontrol.com/static-mcn...,2017-11-13T01:47:26Z,"PSBs want large banks to lend to corporates, r...",Moneycontrol,"\n \n 1. Govt says ONGC, HPCL merger to close ..."
656,"{'id': None, 'name': 'Moneycontrol'}",NaN,News Live: Auto component maker Sandhar Techno...,This blog will keep track of key global and lo...,http://www.moneycontrol.com/news/business/news...,http://static-news.moneycontrol.com/static-mcn...,2017-11-13T01:47:26Z,Auto component maker Sandhar Technologies plan...,Moneycontrol,"\n \n 1. Govt says ONGC, HPCL merger to close ..."
657,"{'id': None, 'name': 'Moneycontrol'}",NaN,News Live: Centre shortlists 18 companies for ...,This blog will keep track of key global and lo...,http://www.moneycontrol.com/news/business/news...,http://static-news.moneycontrol.com/static-mcn...,2017-11-13T01:47:26Z,Centre shortlists 18 companies for strategic s...,Moneycontrol,"\n \n 1. Govt says ONGC, HPCL merger to close ..."
658,"{'id': None, 'name': 'Moneycontrol'}",NaN,"News Live: Idea, Vodafone approve sale of towe...",This blog will keep track of key global and lo...,http://www.moneycontrol.com/news/business/news...,http://static-news.moneycontrol.com/static-mcn...,2017-11-13T01:47:26Z,"Idea, Vodafone approve sale of tower business ...",Moneycontrol,"\n \n 1. Govt says ONGC, HPCL merger to close ..."
659,"{'id': None, 'name': 'Moneycontrol'}",NaN,News Live: RCom stock down 9% on reports of di...,This blog will keep track of key global and lo...,http://www.moneycontrol.com/news/business/news...,http://static-news.moneycontrol.com/static-mcn...,2017-11-13T01:47:26Z,Differences with lenders may hit RCom’s strate...,Moneycontrol,"\n \n 1. Govt says ONGC, HPCL merger to close ..."
...,...,...,...,...,...,...,...,...,...,...
426097,"{'id': None, 'name': 'Moneycontrol'}",NaN,Parliament Winter Session LIVE Updates: Govt t...,Winter Session LIVE Updates: The government is...,https://www.moneycontrol.com/news/politics/par...,https://images.moneycontrol.com/static-mcnews/...,2021-11-29T04:59:49Z,"November 29, 2021 / 10:18 AM IST\r\nParliament...",Moneycontrol,"\n November 29, 2021 / 04:39 PM IST\n ..."
456442,"{'id': None, 'name': 'Moneycontrol'}",NaN,Market LIVE Updates: Indices open gap-down on ...,<ol><li>Market LIVE Updates: Indices open gap-...,https://www.moneycontrol.com/news/business/mar...,https://images.moneycontrol.com/static-mcnews/...,2022-02-14T03:37:45Z,"February 14, 2022 / 08:54 AM IST\r\nCryptocurr...",Moneycontrol,"IndexPricesChangeChange%Sensex65,190.85-206.77..."
458732,"{'id': None, 'name': 'Moneycontrol'}",NaN,Russia Ukraine Conflict LIVE Upates | Russian ...,"Russia Ukraine Conflict LIVE Upates | ""Units o...",https://www.moneycontrol.com/news/world/russia...,https://images.moneycontrol.com/static-mcnews/...,2022-02-18T00:48:29Z,"February 18, 2022 / 06:32 AM IST\r\nRussia Ukr...",Moneycontrol,"\n February 18, 2022 / 10:59 PM IST\n ..."
464114,"{'id': None, 'name': 'Moneycontrol'}",NaN,Russia Ukraine War News LIVE Updates | Janet Y...,Russia Ukraine War News LIVE Updates | Yellen ...,https://www.moneycontrol.com/news/world/russia...,https://images.moneycontrol.com/static-mcnews/...,2022-03-03T01:39:30Z,Russia Ukraine War News LIVE Updates | The Un...,Moneycontrol,"\n March 03, 2022 / 11:01 PM IST\n ..."


In [129]:
empty_rows = TorontoStar[TorontoStar['content_full'] == '']
len(empty_rows)

# print(len(CryptoSlate[CryptoSlate['content_full'] == '']))

5

In [111]:
empty_rows['url'].iloc[00]

'http://www.wsj.com/video/bitcoin-the-worlds-most-dramatic-bubble-ever/518F37A8-B7B4-43AC-869E-6F0B3568EAB6.html'

In [124]:
empty_rows = without[without['content_full'] == '']
len(empty_rows)

231

In [120]:
without['url'].iloc[120]

'https://hindi.moneycontrol.com/news/foreign/indian-siblings-making-30000-dollar-a-month-by-mining-cryptocurrency_286419.html'

In [101]:
empty_rows['url'].iloc[0]

'http://www.moneycontrol.com/news/business/news-live-psbs-want-large-banks-to-lend-to-corporates-retail-only-smaller-banks-says-report-2436469.html'

In [126]:
with2

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
655,"{'id': None, 'name': 'Moneycontrol'}",NaN,News Live: PSBs want large banks to lend to co...,This blog will keep track of key global and lo...,http://www.moneycontrol.com/news/business/news...,http://static-news.moneycontrol.com/static-mcn...,2017-11-13T01:47:26Z,"PSBs want large banks to lend to corporates, r...",Moneycontrol,"\n \n 1. Govt says ONGC, HPCL merger to close ..."
656,"{'id': None, 'name': 'Moneycontrol'}",NaN,News Live: Auto component maker Sandhar Techno...,This blog will keep track of key global and lo...,http://www.moneycontrol.com/news/business/news...,http://static-news.moneycontrol.com/static-mcn...,2017-11-13T01:47:26Z,Auto component maker Sandhar Technologies plan...,Moneycontrol,"\n \n 1. Govt says ONGC, HPCL merger to close ..."
657,"{'id': None, 'name': 'Moneycontrol'}",NaN,News Live: Centre shortlists 18 companies for ...,This blog will keep track of key global and lo...,http://www.moneycontrol.com/news/business/news...,http://static-news.moneycontrol.com/static-mcn...,2017-11-13T01:47:26Z,Centre shortlists 18 companies for strategic s...,Moneycontrol,"\n \n 1. Govt says ONGC, HPCL merger to close ..."
658,"{'id': None, 'name': 'Moneycontrol'}",NaN,"News Live: Idea, Vodafone approve sale of towe...",This blog will keep track of key global and lo...,http://www.moneycontrol.com/news/business/news...,http://static-news.moneycontrol.com/static-mcn...,2017-11-13T01:47:26Z,"Idea, Vodafone approve sale of tower business ...",Moneycontrol,"\n \n 1. Govt says ONGC, HPCL merger to close ..."
659,"{'id': None, 'name': 'Moneycontrol'}",NaN,News Live: RCom stock down 9% on reports of di...,This blog will keep track of key global and lo...,http://www.moneycontrol.com/news/business/news...,http://static-news.moneycontrol.com/static-mcn...,2017-11-13T01:47:26Z,Differences with lenders may hit RCom’s strate...,Moneycontrol,"\n \n 1. Govt says ONGC, HPCL merger to close ..."
...,...,...,...,...,...,...,...,...,...,...
426097,"{'id': None, 'name': 'Moneycontrol'}",NaN,Parliament Winter Session LIVE Updates: Govt t...,Winter Session LIVE Updates: The government is...,https://www.moneycontrol.com/news/politics/par...,https://images.moneycontrol.com/static-mcnews/...,2021-11-29T04:59:49Z,"November 29, 2021 / 10:18 AM IST\r\nParliament...",Moneycontrol,"\n November 29, 2021 / 04:39 PM IST\n ..."
456442,"{'id': None, 'name': 'Moneycontrol'}",NaN,Market LIVE Updates: Indices open gap-down on ...,<ol><li>Market LIVE Updates: Indices open gap-...,https://www.moneycontrol.com/news/business/mar...,https://images.moneycontrol.com/static-mcnews/...,2022-02-14T03:37:45Z,"February 14, 2022 / 08:54 AM IST\r\nCryptocurr...",Moneycontrol,"IndexPricesChangeChange%Sensex65,190.85-206.77..."
458732,"{'id': None, 'name': 'Moneycontrol'}",NaN,Russia Ukraine Conflict LIVE Upates | Russian ...,"Russia Ukraine Conflict LIVE Upates | ""Units o...",https://www.moneycontrol.com/news/world/russia...,https://images.moneycontrol.com/static-mcnews/...,2022-02-18T00:48:29Z,"February 18, 2022 / 06:32 AM IST\r\nRussia Ukr...",Moneycontrol,"\n February 18, 2022 / 10:59 PM IST\n ..."
464114,"{'id': None, 'name': 'Moneycontrol'}",NaN,Russia Ukraine War News LIVE Updates | Janet Y...,Russia Ukraine War News LIVE Updates | Yellen ...,https://www.moneycontrol.com/news/world/russia...,https://images.moneycontrol.com/static-mcnews/...,2022-03-03T01:39:30Z,Russia Ukraine War News LIVE Updates | The Un...,Moneycontrol,"\n March 03, 2022 / 11:01 PM IST\n ..."


In [128]:
Moneycontrol = pd.concat([with1, with2, empty_rows], ignore_index=True)

Moneycontrol

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
0,"{'id': None, 'name': 'Moneycontrol'}",NaN,What changed for the market while you were sle...,A look at top cues from domestic and internati...,http://www.moneycontrol.com/news/business/what...,http://static-news.moneycontrol.com/static-mcn...,2017-11-13T02:23:35Z,"The Nifty, which opened with a slight gap on t...",Moneycontrol,"\nThe Nifty, which opened with a slight gap on..."
1,"{'id': None, 'name': 'Moneycontrol'}",NaN,Watch | Markets@Moneycontrol: New India listin...,A quick check on what could trigger the market...,http://www.moneycontrol.com/news/business/mark...,http://static-news.moneycontrol.com/static-mcn...,2017-11-13T03:12:40Z,- After a steady run in the first week of Nove...,Moneycontrol,\n- After a steady run in the first week of No...
2,"{'id': None, 'name': 'Moneycontrol'}",NaN,Think bitcoin#39;s getting expensive? Try Zimb...,Some are turning to bitcoin out of desperation...,http://www.moneycontrol.com/news/world/think-b...,http://static-news.moneycontrol.com/static-mcn...,2017-11-13T08:11:02Z,"For most investors around the world, bitcoin i...",Moneycontrol,...
3,"{'id': None, 'name': 'Moneycontrol'}",NaN,What changed for the market while you were sle...,A look at top cues from domestic and internati...,http://www.moneycontrol.com/news/business/what...,http://static-news.moneycontrol.com/static-mcn...,2017-11-14T02:18:40Z,The Nifty started on a muted note on Monday an...,Moneycontrol,\nThe Nifty started on a muted note on Monday ...
4,"{'id': None, 'name': 'Moneycontrol'}",NaN,Watch | Markets@Moneycontrol: Khadim to debut ...,A quick check on what could trigger the market...,http://www.moneycontrol.com/news/business/mark...,http://static-news.moneycontrol.com/static-mcn...,2017-11-14T03:11:22Z,The Nifty slipped nearly 100 points and that i...,Moneycontrol,\n-The Nifty slipped nearly 100 points and tha...
...,...,...,...,...,...,...,...,...,...,...
2787,"{'id': None, 'name': 'Moneycontrol'}",Jitendra Kumar Gupta,KEC International: Capitalising on the upswing...,"The company is driven by a robust order book, ...",https://www.moneycontrol.com/news/business/mon...,https://images.moneycontrol.com/static-mcnews/...,2023-02-06T06:11:33Z,Moneycontrol Pro Panorama | Budget 2023: Bonds...,Moneycontrol,
2788,"{'id': None, 'name': 'Moneycontrol'}",Financial Times,Chinaâs recovery might be a bit less than me...,Investors should not assume this upturn in eco...,https://www.moneycontrol.com/news/opinion/chin...,https://images.moneycontrol.com/static-mcnews/...,2023-02-06T06:17:20Z,Moneycontrol Pro Panorama | Budget 2023: Bonds...,Moneycontrol,
2789,"{'id': None, 'name': 'Moneycontrol'}",Anubhav Sahu,Zydus Lifesciences: Transdermal opportunity to...,The US and domestic formulation businesses hel...,https://www.moneycontrol.com/news/business/mon...,https://images.moneycontrol.com/static-mcnews/...,2023-02-06T06:35:11Z,Moneycontrol Pro Panorama | Budget 2023: Bonds...,Moneycontrol,
2790,"{'id': None, 'name': 'Moneycontrol'}",Maryam Farooqui,"IPL 2023: Digital surpasses TV, contributes 58...",The ad pie for both TV and digital will need t...,https://www.moneycontrol.com/news/trends/enter...,https://images.moneycontrol.com/static-mcnews/...,2023-06-06T10:28:14Z,Digital surpassed TV as it contributed 58 perc...,Moneycontrol,


In [130]:
TorontoStar.to_csv(r'C:\Users\user\boyu571_github\01_Kakaobank_SKKU_Research_23\TorontoStar_81.csv', encoding='utf-8-sig')

In [ ]:
#article-13715 > div.post__content-wrapper > div.post-content